<a href="https://colab.research.google.com/github/cjcalderon9804/airbnb-sentiment-analysis/blob/main/airbnb_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An&aacute;lisis de sentimientos para rese&ntilde;as de Airbnb

El presente proyecto aborda una de las aplicaciones más interesantes al momento de trabajar con el **Procesamiento del Lenguage Natural (PLN)** con Python, que consiste en el **Análisis de Sentimientos** aplicado a un conjunto de datos reales de muestra correspondientes a 620 reseñas otorgadas por huéspedes a un total de 10 alojamientos listados dentro de la plataforma de alojamientos temporales *Airbnb* para la íconica ciudad turística ecuatoriana con playas únicas y gente muy acogedora que es **Montañita**.

Este proyecto cumple con los lineamientos de estilo y formato de código dictados en el manual ***PEP-8***.

## ↓↓ Inicio de librer&iacute;as temporales ↓↓

In [1]:
# !pip install flake8 pycodestyle_magic
# %load_ext pycodestyle_magic
# %pycodestyle_on

## ↑↑ Fin de librer&iacute;as temporales ↑↑

# Preprocesamiento inicial de la informaci&oacute;n

## Instalaci&oacute;n e importaci&oacute;n de librer&iacute;as necesarias

In [2]:
!pip install emoji

In [3]:
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path

Se configura pandas para evitar que trunque el texto de las entradas de cada una de las columnas. Adicionalmente, se cambia el directorio base con la librer&iacute;a ***os*** para una mayor facilidad de trabajo. En el directorio base est&aacute;n contenidas las rese&ntilde;as dejadas por hu&eacute;spedes de un total de 10 alojamientos diferentes ubicados en la ciudad de Monta&ntilde;ita, Ecuador, con corte a la fecha del viernes, 06 de enero de 2024.

In [4]:
pd.set_option('display.max_colwidth', None)

In [5]:
directorio = "/content/drive/MyDrive/CDE_CLASES/proyecto_final/abb_reviews"
os.chdir(directorio)
# Verificación de cambio de directorio
print(f"Directorio actual: {os.getcwd()}")

Directorio actual: /content/drive/MyDrive/CDE_CLASES/proyecto_final/abb_reviews


Se obtiene un listado con todos los arhivos correspondientes a las rese&ntilde;as de 10 alojamientos en total.

In [6]:
# Se define un patrón para buscar archivos txt en el directorio
patron = '*.txt'

# Se usa glob para obtener la lista de archivos que coinciden con el patrón
abb_reviews = glob.glob(os.path.join(directorio, patron))

# Se ordenan las reseñas en orden alfabético
abb_reviews = sorted(
    abb_reviews,
    key=lambda x: int(Path(x).stem.split('_')[-1]))

## Conteo de rese&ntilde;as por cada archivo

In [7]:
total_resenas = 0
for archivo in abb_reviews:
    archivo_path = Path(archivo)
    with open(archivo_path, mode="r", encoding="utf-8") as file:
        foo = file.read()
        print(f'Número de reseñas en {archivo_path.name}:\
         {foo.count("Calificación: ")}')
        total_resenas += foo.count("Calificación: ")

print(f"\nTotal de reseñas en {len(abb_reviews)} archivos: {total_resenas}")

Número de reseñas en abb_reviews_00.txt:         9
Número de reseñas en abb_reviews_01.txt:         17
Número de reseñas en abb_reviews_02.txt:         141
Número de reseñas en abb_reviews_03.txt:         15
Número de reseñas en abb_reviews_04.txt:         38
Número de reseñas en abb_reviews_05.txt:         8
Número de reseñas en abb_reviews_06.txt:         200
Número de reseñas en abb_reviews_07.txt:         29
Número de reseñas en abb_reviews_08.txt:         22
Número de reseñas en abb_reviews_09.txt:         141

Total de reseñas en 10 archivos: 620


## Funci&oacute;n para extraer las rese&ntilde;as de hu&eacute;spedes de todos los archivos

Dado que la informaci&oacute;n recopilada no est&aacute; previamente estructurada, es decir, hay comentarios con atributos diferentes a otros, tales como, por ejemplo, "*Se qued&oacute; unas semanas*" o "*Con mascota*", mismos que son eliminados para evitar que ciertas l&iacute;neas con cadenas de texto puedan ser err&oacute;neamente interpretadas como comentarios.

In [8]:
def extraer_comentarios(mi_archivo):
    comentarios = []
    with open(mi_archivo, mode="r", encoding="utf-8") as file:
        texto = file.read()

    # Dividir el texto en fragmentos usando "Calificación:" como delimitador
    fragmentos = texto.split("Calificación:")[1:]

    # Eliminación de fragmentos vacíos
    fragmentos = [
        fragmento.strip() for fragmento in fragmentos if fragmento.strip()
    ]
    for fragmento in fragmentos:
        aux = (
            fragmento.replace(",·", "")
            .replace("Se quedó unas semanas", "")
            .replace("Estadía de una semana", "")
            .replace("Con mascota", "")
        )
        aux = "\n".join(
            linea.strip() for linea in aux.splitlines() if linea.strip()
        )
        comentarios.append(aux.splitlines()[2])

    return comentarios

Se crea una lista para en ella guardar todas las rese&ntilde;as (que a su vez est&aacute;n en listas) para despu&eacute;s guardarlas en un dataframe junto con las calificaciones.

In [9]:
lista_reviews = []

In [10]:
for review_file in abb_reviews:
    lista_reviews.append(extraer_comentarios(review_file))

Se verifica con la funci&oacute;n ***len*** que existan efectivamente ***620*** rese&ntilde;as en total, lo cual se cumple, y posteriormente se crea un archvo de texto plano para verificar los comentarios.

In [11]:
print(len(sum(lista_reviews, [])))

620


Se guardan todas las rese&ntilde;as en un archivo de texto para comprobar de manera manual mediante un editor de textos externo que cada una de las 620 l&iacute;neas correspondientes a comentarios son efectivamente comentarios y no otros de los atributos de las rese&ntilde;as.

In [12]:
if not os.path.exists("comentarios/coment_ALL.txt"):
    with open("comentarios/coment_ALL.txt", mode="w") as file:
        file.write("\n".join(sum(lista_reviews, [])))

## Funci&oacute;n para extraer las calificaciones de hu&eacute;spedes de todos los archivos

Adicionalmente, para el an&aacute;lisis de sentimientos es importante extraer la calificaci&oacute;n del 1 al 5 que dieron los usuarios en cada una de las rese&ntilde;as para contrastarlas con el resultado del an&aacute;lisis de sentimientos. Al igual que con los comentarios en s&iacute;, las puntuaciones se almacenan tambi&eacute;n en formato de listas para despu&eacute;s crear un *DataFrame* de *Pandas* con estas.

In [13]:
def extraer_calificaciones(mi_archivo):
    stars = []
    with open(mi_archivo, mode="r", encoding="utf-8") as file:
        texto = file.read()
        fragmentos = texto.split("Calificación: ")

    for i in range(1, len(fragmentos)):
        stars.append(int(fragmentos[i][0]))

    return stars

In [14]:
lista_stars = []

for review_file in abb_reviews:
    lista_stars.append(extraer_calificaciones(review_file))

Se verifica que la cantidad de calificaciones sea la misma que de comentarios.

In [15]:
len(sum(lista_stars, []))

620

Se crean los &iacute;ndices para identificar a cada uno de los 10 alojamientos previo a la creaci&oacute;n del *DataFrame* de *Pandas*. As&iacute;mismo, se unen todos los elementos de rese&ntilde;as y n&uacute;mero de estrellas de cada alojamiento en una &uacute;nica lista para cada caso.

In [16]:
def unir_listas(lista):
    return sum(lista, [])

In [17]:
numero_resenas = []

In [18]:
for resena in lista_reviews:
    numero_resenas.append(len(resena))

In [19]:
numero_resenas

[9, 17, 141, 15, 38, 8, 200, 29, 22, 141]

In [20]:
id_alojamiento = []
for index, i in enumerate(numero_resenas, start=0):
    id_alojamiento.append(list(np.full(i, index)))

id_alojamiento = sum(id_alojamiento, [])

In [21]:
lista_reviews = unir_listas(lista_reviews)
lista_stars = unir_listas(lista_stars)

## Construcci&oacute;n de DataFrame de Pandas

In [22]:
df_reviews = pd.DataFrame({
    "id_alojamiento": id_alojamiento,
    "review": lista_reviews,
    "stars": lista_stars,
})

Se realiza un muestreo a fin de verificar que el *DataFrame* fue construido de manera correcta y se procede a guardar el mismo en formato *csv* para uso futuro.

In [23]:
df_reviews.sample(5)

,id_alojamiento,review,stars
556,9,"De AirBnB was gelegen in een rustige straat zeer dicht bij het bruisende Montañita. Wij hadden een auto gehuurd, waardoor we erg flexibel waren. Openbaar vervoer is echter een goed alternatief, waarvoor Gijs en Liliana ook allerlei informatie hebben liggen. Het appartement is schoon en ruim. Het is opvallend dat het internet erg betrouwbaar is. Alle voorzieningen zijn aanwezig en er zijn duidelijke instructies vanuit Gijs en Liliana voor het appartement en de mogelijkheden in de buurt. Daarbij zijn we gastvrij en betrokken ontvangen en mochten alle vragen gesteld worden. Absoluut een gedenkwaardig verblijf gehad!",5
299,6,Comunizacion eficaz,5
37,2,"I loved this place, I would have stayed forever if it wasn’t booked. I stayed 2 weeks and everything was great. It’s better than the photos; small, cute and cozy. The views are to die for, especially from bed. For the price, this place is 20 out of 10. Just keep in mind it is not a resort, it is simplistic yet perfect for a down to earth traveler. Quiet spot just a few minutes walk into town/beach. The only downfall was I had to buy my own sponge/dish soap, toilet paper and what not… but no problem considering the price. Johnny is great as well!! Thank you so much I hope to come back",5
177,3,place is good I will come again,5
502,9,"Excelente comunicación y atención. La casita es super Fresca, cómoda, con excelente ubicación para descansar y se encuentra cerca de todo.",5


In [24]:
if not os.path.exists("df_reviews.csv"):
    df_reviews.to_csv("df_reviews.csv", index=False)

A continuaci&oacute;n, se procede a traducir las rese&ntilde;as dejadas por los hu&eacute;spedes a los alojamientos con la ayuda de **Google Cloud Translate**, el cual detecta de manera autom&aacute;tica el idioma de los textos de entrada y los traduce al ingl&eacute;s a fin de procesarlos posteriormente al momento de realizar el an&aacute;lisis de sentimientos.

Para las ejecuciones posteriores del presente *Jupyter Notebook*, se importa el archivo con las rese&ntilde;as ya traducidas a fin de no exceder la cuota de traducciones permitidas en la capa gratuita de *Google Cloud Translate*.

In [25]:
df_traducciones = pd.DataFrame()

In [26]:
from google.cloud import translate_v2 as translate

# Lectura de llave privada de la API de Google Cloud Translate
client = translate.Client.from_service_account_json(
    "../private_key_translate.json"
)

if not os.path.exists("df_reviews_traducido.xlsx"):
    # Función para traducir un texto
    def translate_text(text, target_language='en'):
        result = client.translate(text, target_language=target_language)
        return result['translatedText']

    # Traducción de texto en la columna "review"
    df_reviews['texto_traducido'] = df_reviews["review"].apply(
        lambda x: translate_text(x, target_language="en")
    )

    # Almacenamiento de reseñas traducidas para uso futuro
    df_reviews[[
            "id_alojamiento", "review", "texto_traducido", "stars"
        ]].to_excel(
        "df_reviews_traducido.xlsx", index=False
    )

else:
    df_reviews['texto_traducido'] = pd.read_excel(
        "df_reviews_traducido.xlsx"
    )["texto_traducido"]

Se obtiene una muestra aleatoria de los datos a fin de verificar las traducciones realizadas.

In [27]:
df_reviews.sample(10)

,id_alojamiento,review,stars,texto_traducido
317,6,Espacio perfecto para una escapada,4,Perfect space for a getaway
587,9,"El apartamento es un excelente lugar para estar relajado. Es un lugar muy cómodo y preparado para una estadía muy amena. Gijs y Liliana se preocuparon mucho por que estemos a gusto e incluso prepararon un croquis con la información de todo tipo de lugares cercanos, así como una guía de que hacer y dónde en caso de que queramos hacer otro tipo de actividades. Definitivamente, es un lugar al que me gustaría volver.",5,"The apartment is a great place to relax. It is a very comfortable place and prepared for a very pleasant stay. Gijs and Liliana were very concerned about making sure we were comfortable and even prepared a map with information on all kinds of nearby places, as well as a guide on what to do and where in case we wanted to do other types of activities. It is definitely a place I would like to return to."
473,8,"Uitstekende locatie voor een verblijf in Montanita. Rustige, schone, relaxte en mooie locatie net buiten het centrum. Heerlijk ontbijtje en leuke en behulpzame eigenaren. Aanrader!",5,"Excellent location for a stay in Montanita. Quiet, clean, relaxed and beautiful location just outside the center. Delicious breakfast and nice and helpful owners. Recommended!"
501,9,"the place is amazing, I'm in love wi the beach, Gina is incredible host",5,"the place is amazing, I'm in love wi the beach, Gina is incredible host"
29,2,We felt like home in this beautiful and clean place! The wifi works amazing and the view is wonderful. It is close to everything and the host is very friendly too. Definitely recommend this place.,5,We felt like home in this beautiful and clean place! The wifi works amazing and the view is wonderful. It is close to everything and the host is very friendly too. Definitely recommend this place.
409,6,"amazing place! beautiful interior. has a stove, fridge, bathtub/shower and a.c. it's a minute walk to the center of montanita. highly recommended:)",5,"amazing place! beautiful interior. has a stove, fridge, bathtub/shower and a.c. it's a minute walk to the center of montanita. highly recommended:)"
567,9,"El lugar es espectacular muy acogedor y Liliana muy amable y atenta, le agradezco por que fue mi primer experiencia en Arbnb y fue muy satisfactoria.",5,"The place is spectacular, very cozy and Liliana is very friendly and attentive, I thank her because it was my first experience at Arbnb and it was very satisfactory."
316,6,can't find a better place than this in Montanita!,5,can't find a better place than this in Montanita!
329,6,Place was exactly as described. Host was most friedly and communication was quick and efficient. Great place to stay.,5,Place was exactly as described. Host was most friedly and communication was quick and efficient. Great place to stay.
395,6,"Muy bien servicio, hospitalidad y excelente ubicación.",4,"Very good service, hospitality and excellent location."


# Preparaci&oacute;n de los datos para el Análisis de Sentimientos

##: Elminaci&oacute;n de signos de puntuaci&oacute;n, art&iacute;culos y palabras innecesarias:

Se importa la librer&iacute;a **nltk** detectar y eliminar de los comentarios dejados por los hu&eacute;spedes y traducidos previamente al ingl&eacute;s, los art&iacute;culos, palabras y signos de puntuaci&oacute;n que no van a ser de utilidad al momento de realizar el an&aacute;lisis de sentimientos, y se almacena en una nueva columna del *DataFrame* llamada "*texto_sin_stopwords*".

In [28]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')


def remover_articulos(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
df_reviews["texto_sin_stopwords"] = df_reviews["texto_traducido"].apply(
    lambda x: remover_articulos(x)
)

Se ocupan adem&aacute;s expresiones regulares para dar tratamientos tanto a los espacios sobrantes como a los signos de puntuaci&oacute;n y caracteres innecesarios para el an&aacute;lisis de sentimientos. Adem&aacute;s se preparan los datos para el tratamiento de emojis.

In [30]:
import re


def extraer_no_letras(texto):
    return list(set(re.sub('[a-zA-Z0-9]', '', texto)))

In [31]:
df_reviews["no_texto"] = df_reviews["texto_sin_stopwords"].apply(
    lambda x: extraer_no_letras(x)
)

In [32]:
non_text_chars = list(set(df_reviews["no_texto"].sum()))
non_text_chars.remove(" ")
df_reviews.drop(["no_texto"], axis=1, inplace=True)

## Tratamiento de emojis

Se extrae de la variable definida anteriormente "*non_text_chars*", una lista de todos los emojis presentes en las rese&ntilde;as otorgadas por los hu&eacute;spedes.

Adicionalmente, se asigna una palabra en ingl&eacute;s para dar una interpretaci&oacute;n adecuada a cada uno de estos.

Por &uacute;ltimo, se reemplazan los emojis con sus equivalentes en palabras.

In [33]:
import emoji

emojis_list = [i for i in non_text_chars if emoji.is_emoji(i)]
emojis_dict = {
    '✨': 'sparkling',
    '🙃': 'sarcastic',
    '💖': 'beautiful',
    '😔': 'sad',
    '🙌': 'celebration',
    '😘': 'affectionate',
    '⭐': 'star',
    '👏': 'applause',
    '💕': 'affectionate',
    '😊': 'happy',
    '😍': 'admiration',
    '💓': 'emotion',
    '❣': 'emotion',
    '💜': 'affection',
    '🙈': 'playful',
    '🤗': 'friendly',
    '👌': 'approval',
    '🏻': '',
    '🍻': 'celebration',
    '🌞': 'cheerful',
    '❤': 'love',
    '♥': 'love',
    '💯': 'perfect',
    '👍': 'approval'
}

In [34]:
non_emoji_chars = [i for i in non_text_chars if not emoji.is_emoji(i)]

In [35]:
def quitar_simbolos(texto):
    for char in non_emoji_chars:
        texto = texto.replace(char, "")
    return texto

In [36]:
df_reviews["texto_sin_stopwords"] = df_reviews["texto_sin_stopwords"].apply(
    lambda x: quitar_simbolos(x)
)

In [37]:
df_reviews["texto_sin_stopwords"] = df_reviews["texto_sin_stopwords"].apply(
    lambda x: " ".join(x.split())
)

In [38]:
df_reviews[["texto_traducido", "texto_sin_stopwords"]].iloc[[76, 263], :]

,texto_traducido,texto_sin_stopwords
76,"Everything is very nice, safe and cozy😊",Everything nice safe cozy😊
263,A quiet place with a good location. They are pet friendly 😊,quiet place good location pet friendly 😊


In [39]:
def reemplazar_emojis(texto):
    for (key, value) in emojis_dict.items():
        texto = texto.replace(key, f" {value} ")
    return texto

In [40]:
df_reviews["texto_sin_emojis"] = df_reviews["texto_sin_stopwords"].apply(
    lambda x: reemplazar_emojis(x)
)
df_reviews["texto_sin_emojis"] = df_reviews["texto_sin_emojis"].apply(
    lambda x: " ".join(x.split())
)

In [41]:
if not os.path.exists("comentarios/df_limpio_emojis.xlsx"):
    df_reviews.to_excel("comentarios/df_limpio_emojis.xlsx")

# Análisis de Sentimientos